## Notebook to explore and demonstrate capabilities for inverse queries:
Note inverse queries are those where model output and certain inputs are known and other inputs are to be estimated

In [1]:
from GPyOpt.methods import BayesianOptimization
from GPyOpt import Design_space, experiment_design
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# --- Define your problem
def fun(x): return (6*x-2)**2*np.sin(12*x-4)

def errf(x, t=0): 
    fval = fun(x)
    return np.abs(fval-t)

In [ ]:
domain = [{'name': 'var_1', 'type': 'continuous', 'domain': (0,1)}]
constraint = [{'name': 'constr_1', 'constraint': 'speedOfSound(x) - 6.0'}]
feasible_region = Design_space(space = domain, constraints = constraint)
initial_design = experiment_design.initial_design('random', feasible_region, 10)

In [ ]:
xval = np.linspace(0, 1,200)
plt.plot(xval, fun(xval))
#plt.plot(xval, errf(xval))

type of acquisition function to use. 
- ‘EI’, expected improvement. 
- ‘EI_MCMC’, integrated expected improvement (requires GP_MCMC model). 
- ‘MPI’, maximum probability of improvement. 
- ‘MPI_MCMC’, maximum probability of improvement (requires GP_MCMC model). 
- ‘LCB’, GP-Lower confidence bound. 
- ‘LCB_MCMC’, integrated GP-Lower confidence bound (requires GP_MCMC model).

In [ ]:
# --- Solve your problem
myBopt = BayesianOptimization(f=errf, domain=domain, acquisition_type='EI', constraints=constraint)
myBopt.run_optimization(max_iter=25)
myBopt.plot_acquisition()
myBopt.plot_convergence()
myBopt.x_opt

In [ ]:
myBopt.X[myBopt.Y<0.00001]

## Explore Tf model

In [ ]:
import tensorflow as tf
from tensorflow.python import autograph as ag

In [2]:
import sys
sys.path.append('../kchain/')
import kChain as kc

In [4]:
import requests
#URL to interact with build
url_build = 'http://localhost:12345/darpa/aske/kchain/build'

#URL to interact with evaluate service
url_append = 'http://localhost:12345/darpa/aske/kchain/append'

#URL to interact with evaluate service
url_evaluate = 'http://localhost:12345/darpa/aske/kchain/evaluate'

In [5]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "gamma",
                        "type": "double",
                        "value": "1.4"
                    },
                    {
                        "name": "R",
                        "type": "double",
                        "value": "286.0"
                    },
                    {
                        "name": "theta",
                        "type": "double",
                        "value": "3056.0"
                    },
                    {
                      "name": "T",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "a",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "a = tf.math.pow(R * T *  (  1 + ( gamma-1 ) / ( 1 + ( gamma-1 ) *  ( tf.math.pow( theta/T,2) *  tf.math.exp( theta/T ) /  tf.math.pow( tf.math.exp( theta/T ) - 1,2 ))  ) ), 1/2)",
                   "modelName" : "speedOfSound"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/speedOfSound',
 'modelType': 'Physics',
 'trainedState': 0}

build model

evaluate model with arbitrarily chosen inputs

integrate evaluate model call with residual error function

call error function via BayesOpt

## Single unknown input case

In [ ]:
def computeSoS(T):
    evalPacket = {
      "inputVariables": [
        {
          "name": "T",
          "type": "double",
          "value": str(T)
        }
      ],
      "modelName": "SpeedOfSound",
      "outputVariables": [
        {
          "name": "a",
          "type": "double"
        }
      ]
    }
    #print(evalPacket)
    r = requests.post(url_evaluate, json=evalPacket)
    print(r.json())
    rval = float(r.json()['outputVariables'][0]['value'][1:-1])
    return rval

In [ ]:
def errC(x, t, model): 
    fval = model(x)
    return np.abs(fval-t)

In [ ]:
domain = [{'name': 'T', 'type': 'continuous', 'domain': (100,500)}]
fx = lambda x : errC(x[0], t = 380.0, model = computeSoS)
MAXITER = 35

In [ ]:
myBopt = BayesianOptimization(f=fx, domain=domain, acquisition_type='EI')
myBopt.run_optimization(max_iter=MAXITER)
myBopt.plot_convergence()
print(myBopt.x_opt)
print(myBopt.fx_opt)

## Multiple Unknown Input Case 1:

In [ ]:
def computeSoS2(T, gamma):
    evalPacket = {
      "inputVariables": [
        {
          "name": "T",
          "type": "double",
          "value": str(T)
        },
        {
            "name": "gamma",
            "type": "double",
            "value": str(gamma)
        }
      ],
      "modelName": "SpeedOfSound",
      "outputVariables": [
        {
          "name": "a",
          "type": "double"
        }
      ]
    }
    #print(evalPacket)
    r = requests.post(url_evaluate, json=evalPacket)
    #print(r.json())
    rval = float(r.json()['outputVariables'][0]['value'][1:-1])
    return rval

In [ ]:
def errC2(x1, x2, t, model): 
    fval = model(x1, x2)
    return np.abs(fval-t)

In [ ]:
domain = [{'name': 'T', 'type': 'continuous', 'domain': (100,500)},
          {'name': 'gamma', 'type': 'continuous', 'domain': (1.3,1.5)}]
fx = lambda x : errC2(x[0][0], x[0][1], t = 380.0, model = computeSoS2)
MAXITER = 35

In [ ]:
myBopt = BayesianOptimization(f = fx, domain=domain, acquisition_type='EI')
myBopt.run_optimization(max_iter=MAXITER)
myBopt.plot_convergence()
print(myBopt.x_opt)
print(myBopt.fx_opt)

## Multiple Unknown Input Case 2:

In [ ]:
def computeSoS2(X):
    evalPacket = {
      "inputVariables": [
        {
          "name": "T",
          "type": "double",
          "value": str(X[0])
        },
        {
            "name": "gamma",
            "type": "double",
            "value": str(X[1])
        }
      ],
      "modelName": "SpeedOfSound",
      "outputVariables": [
        {
          "name": "a",
          "type": "double"
        }
      ]
    }
    #print(evalPacket)
    r = requests.post(url_evaluate, json=evalPacket)
    #print(r.json())
    rval = float(r.json()['outputVariables'][0]['value'][1:-1])
    return rval

In [ ]:
def errC(x, t, model): 
    fval = model(x)
    return np.abs(fval-t)

In [ ]:
#name doesnt matter in domain as long as ordering is correct
domain = [{'name': 'T', 'type': 'continuous', 'domain': (100,500)},
          {'name': 'gamma', 'type': 'continuous', 'domain': (1.3,1.5)}]
fx = lambda x : errC(x[0], t = 380.0, model = computeSoS2)
MAXITER = 35

In [ ]:
myBopt = BayesianOptimization(f = fx, domain=domain, acquisition_type='EI')
myBopt.run_optimization(max_iter=MAXITER)
myBopt.plot_convergence()
print(myBopt.x_opt)
print(myBopt.fx_opt)

## Multiple Known and Unknown Input Case 1:

In [ ]:
context = {'gamma':1.4}

myBopt = BayesianOptimization(f = fx, domain=domain, acquisition_type='EI')
myBopt.run_optimization(max_iter=MAXITER, context=context)
myBopt.plot_convergence()
print(myBopt.x_opt)
print(myBopt.fx_opt)

# Using KCHAIN evaluateInverse()

In [7]:
inputVar = [
    {
        "name": "T",
        "type": "double",
        "minValue": 100,
        "maxValue": 1000
    }
]
outputVar = [
    {
        "name": "a",
        "type": "double",
        "value": "350.0"
    }
]
mdlName = 'speedOfSound'

ko = kc.kChainModel(debug=False)
r = ko.evaluateInverse(inputVar = inputVar, outputVar = outputVar, mdlName = mdlName)
print(r)

Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError showed up
Found file with Default values
Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError showed up
Found file with Default values
Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError showed up
Found file with Default values
Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError showed up
Found file with Default values
Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError showed up
Found file with Default values
Found nodeDict and funcList
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
AttributeError sho

In [ ]:
ko._getDefaultValues()

In [ ]:
constraints = []
domain = [{'name': 'T', 'type': 'continuous', 'domain': (100,500)},
          {'name': 'gamma', 'type': 'continuous', 'domain': (1.3,1.5)}]
feasible_region = Design_space(space = domain, constraints = constraints)
initial_design = experiment_design.initial_design('random', feasible_region, 1)
initial_design[0]